In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import numpy as np

In [2]:
mnist = tf.keras.datasets.mnist

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [5]:
num_img = [0, 10, 150, 1000]

for n in num_img:
    np.savez('./tests/test_data/test_img_'+str(n), inputs=x_train[n,:,:,:], targets=y_train[n])

In [6]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [7]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

In [8]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [9]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [10]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [11]:
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [12]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Epoch 1, Loss: 0.13490030169487, Accuracy: 95.90833282470703, Test Loss: 0.06273974478244781, Test Accuracy: 97.93000030517578
Epoch 2, Loss: 0.04264363273978233, Accuracy: 98.66500091552734, Test Loss: 0.04965224117040634, Test Accuracy: 98.43000030517578
Epoch 3, Loss: 0.022941285744309425, Accuracy: 99.25, Test Loss: 0.05311853811144829, Test Accuracy: 98.5
Epoch 4, Loss: 0.014209205284714699, Accuracy: 99.50666809082031, Test Loss: 0.06269436329603195, Test Accuracy: 98.12999725341797
Epoch 5, Loss: 0.00985630415380001, Accuracy: 99.6683349609375, Test Loss: 0.06830089539289474, Test Accuracy: 98.25999450683594


In [13]:
tf.saved_model.save(model, "./model_1")

INFO:tensorflow:Assets written to: ./ml_deploy/model_1\assets


INFO:tensorflow:Assets written to: ./ml_deploy/model_1\assets


In [14]:
test_img_0_data = np.load('./tests/test_data/test_img_0.npz')
test_img_10_data = np.load('./tests/test_data/test_img_10.npz')
test_img_150_data = np.load('./tests/test_data/test_img_150.npz')
test_img_1000_data = np.load('./tests/test_data/test_img_150.npz')

In [15]:
test_img_0_data["targets"], model.predict(tf.expand_dims(test_img_0_data["inputs"], axis=0) )

1/1 [==============================] - 0s 138ms/step


(array(5, dtype=uint8),
 array([[-16.904186 ,  -7.5240507, -12.797028 ,  10.434648 , -15.1880665,
          23.266314 , -11.265102 , -12.165691 ,  -1.6101601,  -0.7997578]],
       dtype=float32))

In [16]:
test_img_10_data["targets"], model.predict(tf.expand_dims(test_img_10_data["inputs"], axis=0) )

1/1 [==============================] - 0s 57ms/step


(array(3, dtype=uint8),
 array([[-11.523441 ,   1.191479 , -10.111725 ,  20.55771  , -15.028469 ,
           2.7588878, -19.147858 ,  -9.39563  ,  -4.1029577,   4.372966 ]],
       dtype=float32))

In [17]:
test_img_150_data["targets"], model.predict(tf.expand_dims(test_img_150_data["inputs"], axis=0) )

1/1 [==============================] - 0s 54ms/step


(array(4, dtype=uint8),
 array([[-11.586139 ,   0.8064519,  -5.961307 , -10.740945 ,  16.312996 ,
          -7.296178 ,  -8.109695 ,   2.181644 , -13.01488  ,  -2.6902046]],
       dtype=float32))

In [18]:
test_img_1000_data["targets"], model.predict(tf.expand_dims(test_img_1000_data["inputs"], axis=0) )

1/1 [==============================] - 0s 49ms/step


(array(4, dtype=uint8),
 array([[-11.586139 ,   0.8064519,  -5.961307 , -10.740945 ,  16.312996 ,
          -7.296178 ,  -8.109695 ,   2.181644 , -13.01488  ,  -2.6902046]],
       dtype=float32))